In [15]:
import numpy as np
x = np.random.random((2,3))
rowmax = x.max(axis = 1, keepdims = True)
x = x - rowmax
exp = np.exp(x)
rowsum = exp.sum(axis = 1,keepdims = True)
print exp / rowsum

x = np.random.random(3)
maxval = x.max()
x = x - maxval
exp = np.exp(x)
sums = exp.sum()
print exp / sums

[[ 0.21801638  0.33018401  0.45179961]
 [ 0.25174817  0.33347858  0.41477324]]
[ 0.23280267  0.35233671  0.41486062]


In [18]:
import numpy as np
import random

def softmax(x):
    """
    Compute the softmax function for each row of the input x.

    It is crucial that this function is optimized for speed because
    it will be used frequently in later code.
    You might find numpy functions np.exp, np.sum, np.reshape,
    np.max, and numpy broadcasting useful for this task. (numpy
    broadcasting documentation:
    http://docs.scipy.org/doc/numpy/user/basics.broadcasting.html)

    You should also make sure that your code works for one
    dimensional inputs (treat the vector as a row), you might find
    it helpful for your later problems.

    You must implement the optimization in problem 1(a) of the 
    written assignment!
    """

    ### YOUR CODE HERE
    
    if(len(x.shape) > 1): # matrix
        rowmax = x.max(axis = 1, keepdims = True)
        x = x - rowmax
        exp = np.exp(x)
        rowsum = exp.sum(axis = 1,keepdims = True)
        return exp / rowsum
        
        
    else:  # vector
        maxval = x.max()
        x = x - maxval
        exp = np.exp(x)
        sums = exp.sum()
        return exp / sums
    
    

Running basic tests...
[ 0.26894142  0.73105858]
[[ 0.26894142  0.73105858]
 [ 0.26894142  0.73105858]]
[[ 0.73105858  0.26894142]]
You should verify these results!

